In [2]:
import sys
sys.path.append('D:/study/brec_ijcnn2022')
import numpy as np
import os

In [4]:
path = 'data/data.npz'
data = np.load(path)
x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']
print("data loaded from", path)

data loaded from data/data.npz


# Preprocess
look up func preprocess() in transformer_model.py
* 16 is sequence length

In [6]:
x_train.shape

(19968, 16, 42)

# y_train
multilabel classification

In [7]:
y_train.shape

(19968, 22)

In [8]:
y_train[0]

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])